In [1]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

## Prefetch

x86 provide prefetch instructions. As a programmer, you may insert ```_mm_prefetch``` in x86 programs to perform software prefetch for your code. The gcc compiler also has a flag ```-fprefetch-loop-arrays``` to automatically insert software prefetch instructions.

### Using prefetch in matrix transpose code

The following example is a highly optimized matrix transpose code. In the example, we try to prefetch the next row.

In [2]:
render_code("./prefetch/transpose.cpp", lang="c++", show=["//START", "//END"])

// ./prefetch/transpose.cpp:42-63 (22 lines)
    //START
    //  Iterate each row.
    f_vector *y_iter = T;
    do{
        //  Iterate each column.
        f_vector *ptr_x = y_iter + block;
        f_vector *ptr_y = y_iter + row_size;

        do{

#ifdef ENABLE_PREFETCH
            _mm_prefetch((char*)(ptr_y + row_size),_MM_HINT_T0);
#endif
            swap_block(ptr_x,ptr_y,block);

            ptr_x += block;
            ptr_y += row_size;
        }while (ptr_y < stop_T);

        y_iter += iter_size;
    }while (y_iter < end);
    //END

Now, let's take a look of what's happening!

In [3]:
! cd prefetch; make clean; make
# ! echo "Without prefetch -- the baseline"; ssh htseng@celebi "lscpu | grep Model; cd courses/CS203/demo/memory/prefetch/; ./transpose"
! echo "Without prefetch -- the baseline"
! lscpu | grep Model
! ./prefetch/transpose

rm -f blockmm_sse blockmm blockmm_sse_prefetch transpose transpose_prefetch
g++ -msse4.1 -mavx -O3 transpose.cpp -o transpose 
g++ -msse4.1 -mavx -O3 -DENABLE_PREFETCH transpose.cpp -o transpose_prefetch 
Without prefetch -- the baseline
Model name:                           13th Gen Intel(R) Core(TM) i7-13700F
Model:                                183
bytes = 1073741824
Starting Data Transpose...   Done
Time: 0.10338 seconds


In [4]:
# ! echo "With prefetch"; ssh htseng@celebi " cd courses/CS203/demo/memory/prefetch/; ./transpose_prefetch"
! echo "With prefetch"
! ./prefetch/transpose_prefetch

With prefetch
bytes = 1073741824
Starting Data Transpose...   Done
Time: 0.097001 seconds


Let's try a different machine now.

In [7]:
#! cd prefetch; make clean; make; lscpu | grep Model
#! echo "Without prefetch -- the baseline"; cd prefetch/; ./transpose
#! echo "With prefetch"; cd prefetch/; ./transpose_prefetch
! ssh htseng@ditto "cd /nfshome/htseng/courses/CSE142/demo/software_optimizations_memory/; make -C ./prefetch clean; make -C ./prefetch ; lscpu | grep Model"
! echo "Without prefetch -- the baseline"; ssh htseng@ditto  "/nfshome/htseng/courses/CSE142/demo/software_optimizations_memory/prefetch/transpose"
! echo "With prefetch";  ssh htseng@ditto  "/nfshome/htseng/courses/CSE142/demo/software_optimizations_memory/prefetch/transpose_prefetch"

make: Entering directory '/nfshome/htseng/courses/CSE142/demo/software_optimizations_memory/prefetch'
rm -f blockmm_sse blockmm blockmm_sse_prefetch transpose transpose_prefetch
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/software_optimizations_memory/prefetch'
make: Entering directory '/nfshome/htseng/courses/CSE142/demo/software_optimizations_memory/prefetch'
g++ -msse4.1 -mavx -O3 transpose.cpp -o transpose 
g++ -msse4.1 -mavx -O3 -DENABLE_PREFETCH transpose.cpp -o transpose_prefetch 
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/software_optimizations_memory/prefetch'
Model:                                80
Model name:                           AMD Ryzen 7 5700G with Radeon Graphics
Without prefetch -- the baseline
bytes = 1073741824
Starting Data Transpose...   Done
Time: 0.124563 seconds
With prefetch
bytes = 1073741824
Starting Data Transpose...   Done
Time: 0.108833 seconds



-- It doesn't work always!